## Evaluation of dhivyeshrk/bart-large-cnn-samsum on samsum

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import transformers 
import datasets 

First load the dataset

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict

df_train = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv')
df_test = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv')
df_val = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv')

df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [3]:
# Convert DataFrames to Hugging Face Datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_val = Dataset.from_pandas(df_val)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
    'validation': dataset_val
})

print(dataset_dict)
dataset_samsum = dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', '__index_level_0__'],
        num_rows: 14731
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


In [4]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', '__index_level_0__'],
        num_rows: 14731
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
!nvidia-smi

Sat Dec 30 08:03:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
model_ckpt = "dhivyeshrk/bart-large-cnn-samsum"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

In [8]:
!pip install rouge_score # rouge is the benchmark score for summarization models

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=802ec01c24dd3190d0c485d452716390263ef192c1e7501efa8948441e6a9bc2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
from datasets import load_metric
from tqdm import tqdm

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements.""" 
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [11]:
def evaluate_summaries(dataset, metric, model, tokenizer, batch_size=16, device=device,
                                   column_text="article",
                                   column_summary="highlights"):
    '''Calculate respective rouge metric for the given data'''
    article_batches = list(chunks(dataset[column_text], batch_size)) # dialogue batches
    target_batches = list(chunks(dataset[column_summary], batch_size))  # target batches
        
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt") # encode the input
            print(type(article_batch))
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),  # generate summary
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8, num_beams=8, max_length=128) 
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True) for s in summaries] # decode them
        
            decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]  # misc processing
            metric.add_batch(predictions=decoded_summaries, references=target_batch)  # add this batch to the metric
        
    score = metric.compute() # Calculate final metric score

    return score

In [12]:
score_fineTuned = evaluate_summaries(dataset_samsum["test"], rouge_metric, model, tokenizer, column_text="dialogue", column_summary="summary", batch_size=8)
rouge_dict_fineTuned = dict((rn, score_fineTuned[rn].mid.fmeasure) for rn in rouge_names)

  0%|          | 0/103 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/103 [00:06<10:41,  6.29s/it]

<class 'list'>


  2%|▏         | 2/103 [00:10<08:06,  4.82s/it]

...

 98%|█████████▊| 101/103 [06:36<00:07,  3.90s/it]

<class 'list'>


 99%|█████████▉| 102/103 [06:40<00:03,  3.90s/it]

<class 'list'>


100%|██████████| 103/103 [06:41<00:00,  3.90s/it]


In [14]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[f"bart-cnn-FineTuned"])

,rouge1,rouge2,rougeL,rougeLsum
bart-cnn-FineTuned,0.392008,0.197664,0.30173,0.30199


In [21]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model. 

In [22]:
model_ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [23]:
score_original = evaluate_summaries(dataset_samsum["test"], rouge_metric, model, tokenizer, column_text="dialogue", column_summary="summary", batch_size=8)
rouge_dict_original = dict((rn, score_original[rn].mid.fmeasure) for rn in rouge_names)

  0%|          | 0/103 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/103 [00:04<08:03,  4.74s/it]

<class 'list'>


  2%|▏         | 2/103 [00:08<07:28,  4.44s/it]

...

 98%|█████████▊| 101/103 [07:49<00:08,  4.50s/it]

<class 'list'>


 99%|█████████▉| 102/103 [07:54<00:04,  4.64s/it]

<class 'list'>


100%|██████████| 103/103 [07:56<00:00,  4.62s/it]


In [24]:
pd.DataFrame.from_records(rouge_dict_original, index=[f"bart-cnn-Facebook"])

,rouge1,rouge2,rougeL,rougeLsum
bart-cnn-Facebook,0.300117,0.097893,0.226542,0.226479


In [25]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Dec 30 08:46:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |  14445MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Now we can test the fine-tuned model

In [43]:
from transformers import pipeline
model_cktpt = 'dhivyeshrk/bart-large-cnn-samsum'
gen_kwargs = {"length_penalty": 0.9, 
             "num_beams": 8,
             "max_length": 64}
# sample_text = dataset_samsum["test"][0]["dialogue"]
# reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model=model_ckpt)

In [46]:
custom_dialogue = """\
Rohit: You look bit down. What’s the matter?
Mahesh: (Sighs) Nothing much.
Rohit: Looks like something isn’t right.
Mahesh: Ya. It’s at the job front. You know that the telecom industry is going through a rough patch because of
        falling prices and shrinking margins. These factors along with consolidation in the industry is
        threatening the stability of our jobs. And even if the job remains, career growth isn’t exciting.
Rohit: I know. I’ve been reading about some of these issues about your industry in the newspapers. 
        So have you thought of any plan?
Mahesh: I’ve been thinking about it for a while, but haven’t concretized anything so far.
Rohit: What have you been thinking, if you can share?
Mahesh: Well, I’ve been thinking of switching to an industry that has at least few decades of growth left.
Rohit: That’s the right approach, but you need to reskill yourself for the industry you’re targeting.
Mahesh: I realize that, and I’ve been leaning toward digital marketing because in that industry I can 
        carry over some of my skills from the current job. Another reason for this inclination 
        is that digital marketing requires far less hardcore technical skills, which will make it relatively 
        easier for me to acquire new skills.
Rohit: Your choice makes sense. So are you thinking of making the transition in near future?
Mahesh: Not immediately. I need to keep the job, as I’ve EMIs to pay. I’m 80-90 percent sure I’ll go with 
        digital marketing as the industry to reskill in, but in the next 2-3 weeks I’ll take more
        opinions on other options, after all I wouldn’t want to change the industry again. 
        And once I finalize the industry, I’ll explore different options to reskill while keeping my current job.
Rohit: Sounds like a plan. If you need I can put you in touch with few friends who can help you finalize 
        your future industry.
"""

# print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])
print(pipe(custom_dialogue)[0]["summary_text"])

The telecom industry is going through a rough patch because of falling prices and shrinking margins. Even if the job remains, career growth isn’t exciting. Mahesh is leaning toward digital marketing because in that industry he can carry over some of his skills from the current job.
